# FFXIV Event Templating Engine
This Jupyter Notebook creates message templates for FFXIV events.\
Each section guides you through the steps, what you need to do to create your own template.\
By default this Notebook comes with a ready made template for the 7th Haven community.\
You can create as many templates as you like, they will all be rendered and you can then just copy paste them to your\
servers and partake as soon as their markdown support releases.

## Setting up
The first 2 sections are just setup to make this Notebook work and create new functionality for the templating engine.\
These functions should not concern you, unless you're experience with Python and Jinja2 and want to create your own template extensions.

In [ ]:
%pip install jinja2 pyyaml

In [ ]:
import jinja2
import datetime
def datetime_filter(value, format='%Y-%m-%d %H:%M%z'):
    dt = datetime.datetime.strptime(value, format)
    unixtime = int(dt.timestamp())
    return unixtime

def add_time(value, duration):
    return value + duration * 60
    

j2_env = jinja2.Environment()
j2_env.filters["hammertime"] = datetime_filter
j2_env.filters["add_time"] = add_time


## Define Templates
Create all templates here that need to be rendered.

E.g.

[
    """\
    # {{ title }}\
    """,\
    """\
    \**{{ title }}**\
    """\
]

will render two templates, one with title as level 1 headline in markdown, and one with the title as bold text.


This notebook comes with template filter extensions for easier handling of dates.

**hammertime** converts a date from the format 2024-12-31 20:00 to a plain number (used in discord timestamps).

This number can then be used to format it in discord e.g. <t:13123:t> just the hours or <t:123123:f> for the full date time.

**add_time** will add a number of minutes to whatever hammertime has been created. This comes handy for reusing date times.\
e.g. {{ somedate | hammertime | add_time(60) }} will add 1 hour to whatever has been somedate originally.

In [ ]:
import jinja2
templates = [
"""
__**{{ event_title }}**__

__**Venue/Event Name**__: {{ venue_name}}
__**Server**__: {{ location.server }}
__**Location**__: {{ location.area }} W{{ location.ward }} P{{ location.plot }}
__**Date and Time**__: <t:{{ date_time | hammertime }}:f>
__**NSFW/Mature Venue/Event?**__: {{ nsfw }}
__**Level of RP immersion**__: {{ rp_level }}
__**Event Description:**__ 
{{ event_description }}

__**DJ's / Music**__
{%- for dj in djs %}
<t:{{ date_time | hammertime | add_time(dj.time_offset) }}:t> [{{ dj.name }}](<https://twitch.tv/{{ dj.twitch_handle }}>)
{%- endfor %}

__**Links**__
:globe_with_meridians: [Website](<{{ links.website }}>)
:discord: [Discord](<https://discord.com/invite/{{ links.discord }}/>)
:instagram: [Instagram](<https://instagram.com/{{ links.instagram }}/>)
:partake: [Partake](<https://partake.gg/events/{{ links.partake }}/>)
""",
"""
# {{ event_title }}
{{ event_description }}

## Location
{{ location.server }} - {{ location.area }} - W{{ location.ward }} - P{{ location.plot }}

## Date and Time
**Doors open at:** <t:{{ date_time | hammertime }}:t>

## DJ Lineup
{%- for dj in djs %}
<t:{{ date_time | hammertime | add_time(dj.time_offset) }}:t> [{{ dj.name }}](<https://twitch.tv/{{ dj.twitch_handle }}>)
{%- endfor %}

## Further Links
:globe_with_meridians: [Website](<{{ links.website }}>)
:discord: [Discord](<https://discord.com/invite/{{ links.discord }}/>)
:instagram: [Instagram](<https://instagram.com/{{ links.instagram }}/>)
:partake: [Partake](<https://partake.gg/events/{{ links.partake }}/>)
"""
]

templates = map(j2_env.from_string, templates)

# Fill the Templates
Fill all the information below to automatically generate texts defined by your templates designed above.

In [ ]:
import yaml

yml_string = """
event_title: It's Valentime, Baby!
venue_name: The Black Wolf
location:
  server: Odin
  area: Mist
  ward: 29
  plot: 2
date_time: 2024-02-14 20:00+0100
nsfw: Yes
rp_level: 3
links:
  website: https://theblackwolf.rocks
  discord: Z2cRA3ygh7
  instagram: theblackwolf.xiv
  partake: 35661
djs:
  - name: DJ Kupo
    time_offset: 0
    twitch_handle: djkupo
  - name: DJ Moogle
    time_offset: 90
    twitch_handle: djmoogle
  - name: DJ Chocobo
    time_offset: 180
    twitch_handle: djchocobo
event_description: |
    This event is gonna be very awesome. With *lot's* of love and shit.
"""

data = yaml.safe_load(yml_string)

for template in templates:
    print(template.render(data))
    print("============")
